# Gender bias w.r.t occupation in contextual embeddings

In [ ]:
!pip install torch transformer-lens datasets

In [ ]:
!pip uninstall numpy -y
!pip install numpy==1.26.4


In [ ]:
#run this block if u want to load bert base
from transformer_lens import HookedEncoder
import torch

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

model2 = HookedEncoder.from_pretrained("google-bert/bert-base-uncased", device="cuda", move_to_device=True)
model2=model2.to(device)
model2.eval()


# text = "The girl is an engineer."
# tokens, token_type_ids, attn_mask = model2.to_tokens(text, move_to_device=True)

# logits, cache = model2.run_with_cache(
#     tokens,
#     token_type_ids=token_type_ids,
#     one_zero_attention_mask=attn_mask,
#     return_type="logits",
# )
# print(list(cache.keys())[:30])

In [ ]:
from transformer_lens import HookedEncoder
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HookedEncoder.from_pretrained("google-bert/bert-large-uncased", device="cuda", move_to_device=True)
model=model.to(device)
model.eval()



In [ ]:
# text = "The girl is an engineer."
# tokens, token_type_ids, attn_mask = model.to_tokens(text, move_to_device=True)

# logits, cache = model.run_with_cache(
#     tokens,
#     token_type_ids=token_type_ids,
#     one_zero_attention_mask=attn_mask,
#     return_type="logits",
# )
# print(list(cache.keys())[:30])

In [ ]:
# def unembed(resid):  # resid: [batch, pos, d_model]
#     return resid @ model.W_U + model.b_U  # -> [batch, pos, d_vocab]

# # 1) Find candidate residual-stream activations
# resid_keys = [k for k in cache.keys() if "resid" in k or "embed" in k]
# for resid_key in resid_keys:
#     print(f"\"{resid_key}\",")    




In [ ]:

# keys_to_probe = [
# ]
# for key in resid_keys:
#     if "hook_normalized_resid_post" in key:
#         keys_to_probe.append(key)    
# print(keys_to_probe)

In [ ]:
# layer_logits = {}
# with torch.no_grad():
#     for k in keys_to_probe:
#         resid = cache[k]
#         layer_logits[k] = unembed(resid)              # logits per layer
#         # probs = layer_logits[k].softmax(dim=-1)     # optional

In [ ]:
import torch.nn.functional as F
prompt = "[MASK] is an engineer."

# Tokenize (if your version supports move_to_device here, use it; else .to(device) manually)
tokens, token_type_ids, attn_mask = model.to_tokens(prompt)
tokens = tokens.to(device)
if token_type_ids is not None: token_type_ids = token_type_ids.to(device)
if attn_mask is not None: attn_mask = attn_mask.to(device)

# Forward pass: logits shape = (batch, pos, d_vocab)
logits, cache = model.run_with_cache(
    tokens,
    token_type_ids=token_type_ids,
    one_zero_attention_mask=attn_mask,
    return_type="logits",
)

# 1) Identify the position you're analyzing
mask_id = model.tokenizer.mask_token_id
mask_pos = (tokens[0] == mask_id).nonzero(as_tuple=False).item()

# 2) Get the distribution at that position
mask_logits = logits[0, mask_pos, :]                # [d_vocab]
mask_probs  = F.softmax(mask_logits, dim=-1)        # [d_vocab]

# 3) Probability of particular candidate tokens (note: BERT WordPiece tokens)
she_id = model.tokenizer.convert_tokens_to_ids("she")
he_id  = model.tokenizer.convert_tokens_to_ids("he")

print("P(she | context) =", float(mask_probs[she_id]))
print("P(he  | context) =", float(mask_probs[he_id]))

# 4) Or just inspect top-k predictions at that position
topk = torch.topk(mask_probs, k=10)
top_tokens = model.tokenizer.convert_ids_to_tokens(topk.indices.tolist())
print(list(zip(top_tokens, topk.values.tolist())))

print("logit(she | context) =", float(mask_logits[she_id]))
print("logit(he  | context) =", float(mask_logits[he_id]))

# 4) Or just inspect top-k predictions at that position
topk = torch.topk(mask_logits, k=10)
top_tokens = model2.tokenizer.convert_ids_to_tokens(topk.indices.tolist())
print(list(zip(top_tokens, topk.values.tolist())))

In [ ]:
def unembed(resid):  # resid: [batch, pos, d_model]
    return resid @ model.W_U + model.b_U  # -> [batch, pos, d_vocab]

layer_keys = ["embed.hook_embed"] + [f"blocks.{l}.hook_resid_post" for l in range(model.cfg.n_layers)]+ [f"blocks.{l}.hook_normalized_resid_post" for l in range(model.cfg.n_layers)] + ["mlm_head.ln.hook_normalized"]
he_id = model.tokenizer.convert_tokens_to_ids("he")
for k in layer_keys:
    resid = cache[k]                               # [batch, pos, d_model]
    layer_logits = unembed(resid)[0, mask_pos, :]   # [d_vocab]
    layer_probs  = F.softmax(layer_logits, dim=-1)

    print(k, "P(she)=", float(layer_probs[she_id]), "P(he)=", float(layer_probs[he_id]))

In [ ]:
import torch.nn.functional as F
prompt = "The [MASK] is an engineer."

# Tokenize (if your version supports move_to_device here, use it; else .to(device) manually)
tokens, token_type_ids, attn_mask = model.to_tokens(prompt)
tokens = tokens.to(device)
if token_type_ids is not None: token_type_ids = token_type_ids.to(device)
if attn_mask is not None: attn_mask = attn_mask.to(device)

# Forward pass: logits shape = (batch, pos, d_vocab)
logits, cache = model.run_with_cache(
    tokens,
    token_type_ids=token_type_ids,
    one_zero_attention_mask=attn_mask,
    return_type="logits",
)

# 1) Identify the position you're analyzing
mask_id = model.tokenizer.mask_token_id
mask_pos = (tokens[0] == mask_id).nonzero(as_tuple=False).item()

# 2) Get the distribution at that position
mask_logits = logits[0, mask_pos, :]                # [d_vocab]
mask_probs  = F.softmax(mask_logits, dim=-1)        # [d_vocab]

# 3) Probability of particular candidate tokens (note: BERT WordPiece tokens)
she_id = model.tokenizer.convert_tokens_to_ids("she")
he_id  = model.tokenizer.convert_tokens_to_ids("he")

print("P(she | context) =", float(mask_probs[she_id]))
print("P(he  | context) =", float(mask_probs[he_id]))

# 4) Or just inspect top-k predictions at that position
topk = torch.topk(mask_probs, k=10)
top_tokens = model.tokenizer.convert_ids_to_tokens(topk.indices.tolist())
print(list(zip(top_tokens, topk.values.tolist())))

In [ ]:
def unembed(resid):  # resid: [batch, pos, d_model]
    return resid @ model.W_U + model.b_U  # -> [batch, pos, d_vocab]

layer_keys = ["embed.hook_embed"] + [f"blocks.{l}.hook_resid_post" for l in range(model.cfg.n_layers)]+ [f"blocks.{l}.hook_normalized_resid_post" for l in range(model.cfg.n_layers)] + ["mlm_head.ln.hook_normalized"]
he_id = model.tokenizer.convert_tokens_to_ids("he")
for k in layer_keys:
    resid = cache[k]                               # [batch, pos, d_model]
    layer_logits = unembed(resid)[0, mask_pos, :]   # [d_vocab]
    layer_probs  = F.softmax(layer_logits, dim=-1)

    print(k, "P(she)=", float(layer_probs[she_id]), "P(he)=", float(layer_probs[he_id]))

# experiment - if the unembedding has gender bias

In [ ]:
import torch
import torch.nn.functional as F

d_model = model.cfg.d_model
he_id   = model.tokenizer.convert_tokens_to_ids("he")

def unembed(x):
    return x @ model.W_U + model.b_U  # x: [batch, pos, d_model] -> [batch, pos, d_vocab] [web:89]

mask_id  = model.tokenizer.mask_token_id
mask_pos = (tokens[0] == mask_id).nonzero(as_tuple=False).item()

# True scalar targets at the [MASK] position
with torch.no_grad():
    true_logit_he = logits[0, mask_pos, he_id].item()
    true_prob_he  = F.softmax(logits[0, mask_pos, :], dim=-1)[he_id].item()

# Candidate activations: only tensors shaped [batch, pos, d_model]
cands = [
    k for k, v in cache.items()
    if isinstance(v, torch.Tensor) and v.ndim == 3 and v.shape[-1] == d_model
]

logit_errs = []
prob_errs  = []

with torch.no_grad():
    for k in cands:
        approx_logits = unembed(cache[k])                # [batch, pos, d_vocab]
        approx_logit_he = approx_logits[0, mask_pos, he_id].item()

        # For probability, softmax at that position
        approx_prob_he = F.softmax(approx_logits[0, mask_pos, :], dim=-1)[he_id].item()

        logit_errs.append((abs(approx_logit_he - true_logit_he), k))
        prob_errs.append((abs(approx_prob_he  - true_prob_he ), k))

logit_errs.sort()
prob_errs.sort()

print("True logit(he):", true_logit_he)
print("Top-10 best matches for logit(he):")
for err, k in logit_errs[:10]:
    print(f"{err:.6g}", k)

print("\nTrue P(he):", true_prob_he)
print("Top-10 best matches for P(he):")
for err, k in prob_errs[:10]:
    print(f"{err:.6g}", k)


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# W_U: [d_model, d_vocab]  [web:89]
W_U = model.W_U.detach()

# Each token corresponds to a column vector W_U[:, token_id]
col_norms = torch.linalg.norm(W_U, dim=0)          # [d_vocab]
col_norms_np = col_norms.cpu().numpy()

# Summary stats
mn = float(col_norms.min().item())
mx = float(col_norms.max().item())
mean = float(col_norms.mean().item())
std = float(col_norms.std(unbiased=False).item())

print("Unembed column-norm stats:")
print("min :", mn)
print("max :", mx)
print("mean:", mean)
print("std :", std)

# (Optional) identify tokens with min/max norms
min_id = int(torch.argmin(col_norms).item())
max_id = int(torch.argmax(col_norms).item())
print("\nMin-norm token:", min_id, model.tokenizer.convert_ids_to_tokens([min_id])[0], mn)
print("Max-norm token:", max_id, model.tokenizer.convert_ids_to_tokens([max_id])[0], mx)

# Plot: histogram is most readable for ~30k tokens
plt.figure(figsize=(7, 4))
plt.hist(col_norms_np, bins=200)
plt.title("BERT unembedding column norms (||W_U[:, token]||)")
plt.xlabel("L2 norm")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


In [ ]:


# norms
norms = col_norms.cpu().numpy()
token_ids = np.arange(len(norms))

# sort by norm
order = np.argsort(norms)
sorted_norms = norms[order]
sorted_ids = token_ids[order]

eps = 5*(1e-5)

clusters = []
current_cluster = [0]

for i in range(1, len(sorted_norms)):
    if abs(sorted_norms[i] - sorted_norms[i-1]) <= eps:
        current_cluster.append(i)
    else:
        clusters.append(current_cluster)
        current_cluster = [i]

clusters.append(current_cluster)

large_clusters = [
    c for c in clusters if len(c) >= 300
]

print(f"Found {len(large_clusters)} large clusters\n")

for i, c in enumerate(large_clusters):
    norm_val = sorted_norms[c[0]]
    print(f"Cluster {i}: size={len(c)}, norm≈{norm_val:.8f}")



In [ ]:
for i, c in enumerate(large_clusters):
    print(f"\n=== Cluster {i} ===")
    print(f"size={len(c)}, norm≈{sorted_norms[c[0]]:.8f}\n")
    count=0
    for idx in c:
        tid = int(sorted_ids[idx])
        tok = model.tokenizer.convert_ids_to_tokens(tid)
        if "unused" not in tok:
            print(f"{tid:5d}  {tok}")
        else:
            count=count+1
    print (f"unused tokens: {count}" )           
            

In [ ]:
# W_U: [d_model, d_vocab]  [web:89]
W_U = model2.W_U.detach()

# Each token corresponds to a column vector W_U[:, token_id]
col_norms = torch.linalg.norm(W_U, dim=0)          # [d_vocab]
col_norms_np = col_norms.cpu().numpy()

# Summary stats
mn = float(col_norms.min().item())
mx = float(col_norms.max().item())
mean = float(col_norms.mean().item())
std = float(col_norms.std(unbiased=False).item())

print("Unembed column-norm stats:")
print("min :", mn)
print("max :", mx)
print("mean:", mean)
print("std :", std)

# (Optional) identify tokens with min/max norms
min_id = int(torch.argmin(col_norms).item())
max_id = int(torch.argmax(col_norms).item())
print("\nMin-norm token:", min_id, model2.tokenizer.convert_ids_to_tokens([min_id])[0], mn)
print("Max-norm token:", max_id, model2.tokenizer.convert_ids_to_tokens([max_id])[0], mx)

# Plot: histogram is most readable for ~30k tokens
plt.figure(figsize=(7, 4))
plt.hist(col_norms_np, bins=200)
plt.title("BERT unembedding column norms (||W_U[:, token]||)")
plt.xlabel("L2 norm")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


In [ ]:
import torch

# W_U: [d_model, d_vocab] so token vectors are columns [web:89]
W_U = model.W_U.detach()  # keep on GPU/CPU, no need to move unless printing a lot

col_norms = torch.linalg.norm(W_U, dim=0)  # [d_vocab]

# Sort tokens by norm
sorted_norms, sorted_ids = torch.sort(col_norms, descending=True)

top_k = 50
bot_k = 50

top_ids = sorted_ids[:top_k].tolist()
bot_ids = sorted_ids[-bot_k:].tolist()

top_toks = model.tokenizer.convert_ids_to_tokens(top_ids)  # id -> token strings [web:163]
bot_toks = model.tokenizer.convert_ids_to_tokens(bot_ids)  # id -> token strings [web:163]

print(f"Top {top_k} tokens by ||W_U[:,tok]||:")
for i, (tok, tid) in enumerate(zip(top_toks, top_ids)):
    print(f"{i+1:>2}. {tok:<15}  id={tid:<6}  norm={sorted_norms[i].item():.6f}")

print(f"\nBottom {bot_k} tokens by ||W_U[:,tok]||:")
# bottom norms are at the end of sorted_norms
for j, (tok, tid) in enumerate(zip(bot_toks, bot_ids)):
    norm_val = col_norms[tid].item()
    print(f"{j+1:>2}. {tok:<15}  id={tid:<6}  norm={norm_val:.6f}")


In [ ]:
import torch

# W_U: [d_model, d_vocab], token vectors are columns [web:89]
W_U = model.W_U.detach()

he_id  = model.tokenizer.convert_tokens_to_ids("he")   # token -> id [web:163]
she_id = model.tokenizer.convert_tokens_to_ids("she")  # token -> id [web:163]

he_norm  = torch.linalg.norm(W_U[:, he_id]).item()
she_norm = torch.linalg.norm(W_U[:, she_id]).item()

print(f"||W_U[:, 'he']||  = {he_norm:.6f}  (id={he_id})")
print(f"||W_U[:, 'she']|| = {she_norm:.6f} (id={she_id})")

In [ ]:
import torch
import pandas as pd

# Unembed matrix
W_U = model.W_U.detach()
tok = model.tokenizer

def is_single_wordpiece(s: str) -> bool:
    pieces = tok.tokenize(s)
    return (len(pieces) == 1) and (pieces[0] == s)

def unembed_col_norm(token_str: str) -> float:
    token_id = tok.convert_tokens_to_ids(token_str)
    if token_id is None or token_id == tok.unk_token_id:
        return float("nan")
    return torch.linalg.norm(W_U[:, token_id]).item()


#used gemini 3 to generate pairs
pairs = [
    ("he","she"), ("him","her"), ("his","hers"), ("himself","herself"),
    ("mr","mrs"), ("sir","madam"),

    ("man","woman"), ("men","women"), ("boy","girl"), ("boys","girls"),
    ("father","mother"), ("dad","mom"), ("daddy","mommy"),
    ("son","daughter"), ("sons","daughters"),
    ("brother","sister"), ("brothers","sisters"),
    ("uncle","aunt"), ("uncles","aunts"),
    ("husband","wife"), ("husbands","wives"),
    ("boyfriend","girlfriend"), ("boyfriends","girlfriends"),

    ("king","queen"), ("kings","queens"),
    ("prince","princess"), ("princes","princesses"),
    ("gentleman","lady"), ("gentlemen","ladies"),

    ("grandfather","grandmother"), ("grandpa","grandma"),
    ("stepfather","stepmother"), ("stepson","stepdaughter"),

    ("policeman","policewoman"),
    ("fireman","firewoman"),
    ("chairman","chairwoman"),
    ("businessman","businesswoman"),
    ("waiter","waitress"),
    ("actor","actress"),
    ("hero","heroine"),
    ("monk","nun"),
    ("groom","bride"),

    ("john","mary"), ("james","jennifer"), ("robert","linda"),
    ("michael","elizabeth"), ("william","barbara"),
    ("david","susan"), ("richard","jessica"),
    ("joseph","sarah"), ("thomas","karen"),
    ("charles","nancy"), ("daniel","lisa"),
    ("matthew","betty"), ("anthony","margaret"),
    ("mark","sandra"), ("paul","ashley"),
    ("steven","kimberly"), ("andrew","emily"),
    ("kenneth","donna"), ("george","michelle"),
]

rows = []
kept_pairs = []

for m, f in pairs:
    if is_single_wordpiece(m) and is_single_wordpiece(f):
        m_id = tok.convert_tokens_to_ids(m)
        f_id = tok.convert_tokens_to_ids(f)

        if m_id != tok.unk_token_id and f_id != tok.unk_token_id:
            m_norm = unembed_col_norm(m)
            f_norm = unembed_col_norm(f)

            rows.append({
                "male_token": m,
                "male_id": m_id,
                "male_norm": m_norm,
                "female_token": f,
                "female_id": f_id,
                "female_norm": f_norm,
                "norm_diff_m_minus_f": m_norm - f_norm
            })
            kept_pairs.append((m, f))

df = pd.DataFrame(rows)

print(f"Kept paired tokens: {len(kept_pairs)}")
assert len(kept_pairs) >= 50, "Need at least 50 valid male↔female pairs."


df_sorted = df.sort_values(
    "norm_diff_m_minus_f",
    key=lambda x: x,
    ascending=False
)

print("\n=== Paired unembed column norms (sorted by |male − female|) ===\n")
for _, r in df_sorted.iterrows():
    print(
        f"{r['male_token']:<14} ({r['male_norm']:.6f})   "
        f"{r['female_token']:<14} ({r['female_norm']:.6f})   "
        f"Δ={r['norm_diff_m_minus_f']:+.6f}"
    )


stats = pd.DataFrame({
    "male": df["male_norm"].describe(),
    "female": df["female_norm"].describe(),
    "male_minus_female": df["norm_diff_m_minus_f"].describe()
})

print("\n=== Summary statistics ===")
print(stats)


df.to_csv("gender_token_unembed_norms_paired.csv", index=False)
print("\nWrote gender_token_unembed_norms_paired.csv")


# expt seeing if cosine similarity of the embeddings show gender bias. 

In [ ]:
import torch
import torch.nn.functional as F

stereo = "The girl is an engineer."
anti   = "The boy is an engineer."

tokens, ttypes, amask = model.to_tokens([stereo, anti])
logits, cache = model.run_with_cache(
    tokens,
    token_type_ids=ttypes,
    one_zero_attention_mask=amask,
    return_type="logits",
)

def find_tok_pos(model, tokens_1seq, tok_str):
    tok_id = model.tokenizer.convert_tokens_to_ids(tok_str)
    # tokens_1seq: [pos]
    matches = (tokens_1seq == tok_id).nonzero(as_tuple=False).flatten()
    assert len(matches) > 0, (tok_str, "not found")
    return int(matches[0])


girl_pos = find_tok_pos(model, tokens[0], "girl")
boy_pos  = find_tok_pos(model, tokens[1], "boy")
eng_pos0 = find_tok_pos(model, tokens[0], "engineer")
eng_pos1 = find_tok_pos(model, tokens[1], "engineer")

layer_keys = ["embed.hook_embed"] + [f"blocks.{l}.hook_resid_post" for l in range(model.cfg.n_layers)]+["mlm_head.ln.hook_normalized"]

cos_by_layer = []
for k in layer_keys:
    h = cache[k]  # [batch, pos, d_model]
    girl_vec = h[0, girl_pos]
    eng_vec0 = h[0, eng_pos0]
    boy_vec  = h[1, boy_pos]
    eng_vec1 = h[1, eng_pos1]

    cos_girl_eng = F.cosine_similarity(girl_vec, eng_vec0, dim=0).item()
    cos_boy_eng  = F.cosine_similarity(boy_vec,  eng_vec1, dim=0).item()
    print((k, cos_girl_eng, cos_boy_eng, cos_girl_eng - cos_boy_eng))
    cos_by_layer.append((k, cos_girl_eng, cos_boy_eng, cos_girl_eng - cos_boy_eng))



In [ ]:
import torch
import torch.nn.functional as F

stereo = "He is an engineer."
anti   = "She is an engineer."

tokens, ttypes, amask = model.to_tokens([stereo, anti])
logits, cache = model.run_with_cache(
    tokens,
    token_type_ids=ttypes,
    one_zero_attention_mask=amask,
    return_type="logits",
)


girl_pos = find_tok_pos(model, tokens[0], "he")
boy_pos  = find_tok_pos(model, tokens[1], "she")
eng_pos0 = find_tok_pos(model, tokens[0], "engineer")
eng_pos1 = find_tok_pos(model, tokens[1], "engineer")


layer_keys = ["embed.hook_embed"] + [f"blocks.{l}.hook_resid_post" for l in range(model.cfg.n_layers)]+["mlm_head.ln.hook_normalized"]

cos_by_layer = []
for k in layer_keys:
    h = cache[k]  # [batch, pos, d_model]
    girl_vec = h[0, girl_pos]
    eng_vec0 = h[0, eng_pos0]
    boy_vec  = h[1, boy_pos]
    eng_vec1 = h[1, eng_pos1]

    cos_girl_eng = F.cosine_similarity(girl_vec, eng_vec0, dim=0).item()
    cos_boy_eng  = F.cosine_similarity(boy_vec,  eng_vec1, dim=0).item()
    print((k, cos_girl_eng, cos_boy_eng, cos_girl_eng - cos_boy_eng))
    cos_by_layer.append((k, cos_girl_eng, cos_boy_eng, cos_girl_eng - cos_boy_eng))

# experiment 4 - gender bias on wino bias dataset measured by cosine similarity between embeddings

In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datasets import load_dataset
from scipy.stats import ttest_rel


In [ ]:
type1_pro  = load_dataset("uclanlp/wino_bias", "type1_pro",  split="test")
type1_anti = load_dataset("uclanlp/wino_bias", "type1_anti", split="test")

type2_pro  = load_dataset("uclanlp/wino_bias", "type2_pro",  split="test")
type2_anti = load_dataset("uclanlp/wino_bias", "type2_anti", split="test")


In [ ]:
assert len(type1_pro) == len(type1_anti)
assert len(type2_pro) == len(type2_anti)


In [ ]:
IGNORE_TOKENS = {"the", "a", "an"}
PRONOUNS = {"he", "she", "him", "her", "his", "hers"}


In [ ]:
import re
# import pandas as pd
# from datasets import load_dataset

IGNORE_TOKENS = {"the", "a", "an"}
PRONOUNS = {"he", "she", "him", "her", "his", "hers"}
DETERMINERS = IGNORE_TOKENS


def parse_coref_indices(ex):
    """
    Winobias coreference_clusters is a string list, e.g. ['3','4','6','6'].
    Treat it as a *single* cluster of indices, remove duplicates, sort.
    """
    raw = ex["coreference_clusters"]
    # raw is something like ['3','4','6','6']
    idxs = sorted({int(i) for i in raw})
    return idxs

def build_filtered_examples(dataset, split_name, model, max_examples=None):
    """
    For one of: type1_pro, type1_anti, type2_pro, type2_anti.

    Returns list of dicts with:
      - split: split_name
      - sentence
      - pron_word, pron_pos
      - occ_word, occ_pos
      - coref_indices (string)
    Constraints:
      - cluster has exactly 1 pronoun and 1 non-determiner entity
      - occupation is single BERT token (WordPiece)
    """
    rows = []
    n_total, n_bad_coref, n_multi_entities, n_bad_tokenization = 0, 0, 0, 0

    for i, ex in enumerate(dataset):
        if max_examples is not None and i >= max_examples:
            break

        n_total += 1
        words = ex["tokens"]
        words_l = [w.lower() for w in words]

        try:
            idxs = parse_coref_indices(ex)
        except Exception:
            n_bad_coref += 1
            continue

        pron_positions = []
        entity_positions = []

        for idx in idxs:
            if idx < 0 or idx >= len(words_l):
                continue
            w = words_l[idx]
            if w in PRONOUNS:
                pron_positions.append(idx)
            elif w in DETERMINERS:
                continue
            else:
                entity_positions.append(idx)

        if len(pron_positions) != 1 or len(entity_positions) != 1:
            n_multi_entities += 1
            continue

        pron_pos = pron_positions[0]
        occ_pos  = entity_positions[0]
        pron_word = words[pron_pos]
        occ_word  = words[occ_pos]

        # BERT tokenization: occupation must be a single token
        occ_pieces = model.tokenizer.tokenize(occ_word)
        if len(occ_pieces) != 1:
            n_bad_tokenization += 1
            continue

        sentence = " ".join(words)

        rows.append(
            {
                "split": split_name,
                "sentence": sentence,
                "pron_word": pron_word,
                "pron_pos": pron_pos,
                "occ_word": occ_word,
                "occ_pos": occ_pos,
                "coref_indices": ";".join(map(str, idxs)),
            }
        )

    print(
        f"[{split_name}] total={n_total}, bad_coref={n_bad_coref}, "
        f"multi/none={n_multi_entities}, bad_tok={n_bad_tokenization}, kept={len(rows)}"
    )
    return rows


# Build filtered rows for all four subsets
rows_type1_pro  = build_filtered_examples(type1_pro,  "type1_pro",  model)
rows_type1_anti = build_filtered_examples(type1_anti, "type1_anti", model)
rows_type2_pro  = build_filtered_examples(type2_pro,  "type2_pro",  model)
rows_type2_anti = build_filtered_examples(type2_anti, "type2_anti", model)

df_type1_pro  = pd.DataFrame(rows_type1_pro)
df_type1_anti = pd.DataFrame(rows_type1_anti)
df_type2_pro  = pd.DataFrame(rows_type2_pro)
df_type2_anti = pd.DataFrame(rows_type2_anti)


In [ ]:
 # Align pro/anti by index within each type, and concatenate into a single paired df
def make_paired_df(df_pro, df_anti, type_name: str):
    n = min(len(df_pro), len(df_anti))
    df_pro  = df_pro.iloc[:n].reset_index(drop=True)
    df_anti = df_anti.iloc[:n].reset_index(drop=True)
    df_pro  = df_pro.rename(columns=lambda c: c + "_pro")
    df_anti = df_anti.rename(columns=lambda c: c + "_anti")
    paired = pd.concat([df_pro, df_anti], axis=1)
    paired["type"] = type_name
    return paired

paired_type1 = make_paired_df(df_type1_pro, df_type1_anti, "type1")
paired_type2 = make_paired_df(df_type2_pro, df_type2_anti, "type2")

paired_df = pd.concat([paired_type1, paired_type2], ignore_index=True)
paired_df.to_csv("winobias_filtered_paired_for_bert.csv", index=False)
print("Saved winobias_filtered_paired_for_bert.csv with", len(paired_df), "pairs")


In [ ]:
layer_keys = ["embed.hook_embed"] + [f"blocks.{l}.hook_resid_post" for l in range(model.cfg.n_layers)]+ [f"blocks.{l}.hook_normalized_resid_post" for l in range(model.cfg.n_layers)] + ["mlm_head.ln.hook_normalized"]

In [ ]:
def batched_layerwise_cos_paired(paired_df, batch_size: int = 32):
    """
    For each layer k, returns two lists of length N_pairs:
      cos_pro[k], cos_anti[k]
    where cos = cos(h_l(pron), h_l(occ)).
    """
    N = len(paired_df)
    cos_pro  = {k: [] for k in layer_keys}
    cos_anti = {k: [] for k in layer_keys}

    for start in range(0, N, batch_size):
        batch = paired_df.iloc[start : start + batch_size]

        # PRO batch
        sent_pro  = batch["sentence_pro"].tolist()
        pron_pro  = batch["pron_pos_pro"].astype(int).tolist()
        occ_pro   = batch["occ_pos_pro"].astype(int).tolist()

        tok_pro, tt_pro, am_pro = model.to_tokens(sent_pro)
        tok_pro = tok_pro.to(device)
        if tt_pro is not None: tt_pro = tt_pro.to(device)
        if am_pro is not None: am_pro = am_pro.to(device)

        _, cache_pro = model.run_with_cache(
            tok_pro,
            token_type_ids=tt_pro,
            one_zero_attention_mask=am_pro,
            return_type="logits",
        )

        B_pro = tok_pro.size(0)
        idx_b_pro = torch.arange(B_pro, device=device)
        pron_idx_pro = torch.tensor(pron_pro, device=device)
        occ_idx_pro  = torch.tensor(occ_pro,  device=device)

        # ANTI batch
        sent_anti = batch["sentence_anti"].tolist()
        pron_anti = batch["pron_pos_anti"].astype(int).tolist()
        occ_anti  = batch["occ_pos_anti"].astype(int).tolist()

        tok_anti, tt_anti, am_anti = model.to_tokens(sent_anti)
        tok_anti = tok_anti.to(device)
        if tt_anti is not None: tt_anti = tt_anti.to(device)
        if am_anti is not None: am_anti = am_anti.to(device)

        _, cache_anti = model.run_with_cache(
            tok_anti,
            token_type_ids=tt_anti,
            one_zero_attention_mask=am_anti,
            return_type="logits",
        )

        B_anti = tok_anti.size(0)
        assert B_pro == B_anti
        idx_b_anti = torch.arange(B_anti, device=device)
        pron_idx_anti = torch.tensor(pron_anti, device=device)
        occ_idx_anti  = torch.tensor(occ_anti,  device=device)

        for k in layer_keys:
            h_pro   = cache_pro[k]   # [B, pos, d_model]
            h_anti  = cache_anti[k]  # [B, pos, d_model]

            pron_vecs_pro = h_pro[idx_b_pro,  pron_idx_pro]
            occ_vecs_pro  = h_pro[idx_b_pro,  occ_idx_pro]
            cos_batch_pro = F.cosine_similarity(pron_vecs_pro, occ_vecs_pro, dim=-1)

            pron_vecs_anti = h_anti[idx_b_anti, pron_idx_anti]
            occ_vecs_anti  = h_anti[idx_b_anti, occ_idx_anti]
            cos_batch_anti = F.cosine_similarity(pron_vecs_anti, occ_vecs_anti, dim=-1)

            cos_pro[k].extend(cos_batch_pro.detach().cpu().tolist())
            cos_anti[k].extend(cos_batch_anti.detach().cpu().tolist())

        print(f"Processed {start + B_pro} / {N} pairs")

    return cos_pro, cos_anti

In [ ]:
cos_pro, cos_anti = batched_layerwise_cos_paired(paired_df, batch_size=64)

In [ ]:
# print (cos_pro,cos_anti)

In [ ]:
#helper function for better prints
import re
def clean_layer_name(name: str) -> str:
    name = str(name)
    if "embed" in name:
        return "Emb"
    if "mlm_head" in name:
        return "Head_norm" if "norm" in name or "normalized" in name else "Head"
    name = name.replace("blocks.", "")
    m = re.search(r"(\d+)", name)
    if not m:
        base = name
    else:
        layer_idx = m.group(1)
        base = f"L{layer_idx}"
    if "norm" in name or "normalized" in name:
        base = base + "_norm"
    return base

res_df["layer"] = res_df["layer"].astype(str).apply(clean_layer_name)
print(res_df)

In [ ]:
#pttest
results = []
from scipy.stats import ttest_rel

for layer_name in layer_keys:
    x = np.array(cos_pro[layer_name], dtype=np.float32)   # pro
    y = np.array(cos_anti[layer_name], dtype=np.float32)  # anti
    assert x.shape == y.shape

    delta = y - x  # anti - pro; <0 means pro (stereotype) higher similarity
    mean_delta = float(delta.mean())

    delta2 = (y - x)*2/(y+x)  # anti - pro; <0 means pro (stereotype) higher similarity

    mean_delta2 = float(delta2.mean())
    n = int(delta.size)

    t_stat, p_t = ttest_rel(y, x)        # tests mean(delta) == 0

    results.append(
        {
            "layer": layer_name,
            "mean_delta": mean_delta,
            "mean_delta2": mean_delta2,
            "p_ttest": float(p_t),
            "n_pairs": n,
        }
    )

res_df = pd.DataFrame(results)
res_df.to_csv("winobias_layerwise_paired_tests.csv", index=False)
res_df["layer"] = res_df["layer"].astype(str).apply(clean_layer_name)
print(res_df)


In [ ]:
import numpy as np
import pandas as pd

def permutation_pvalue_xy(x, y, n_permutations=10000, seed=42):
    """
    Two-sided permutation test for mean(y - x) == 0,
    implemented by shuffling the pooled (x,y) values and
    randomly re-partitioning into two groups of size len(x), len(y)
    each permutation.
    """
    rng = np.random.default_rng(seed)

    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)

    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]

    n_x, n_y = len(x), len(y)
    if n_x < 2 or n_y < 2:
        return np.nan, np.nan

    obs_stat = y.mean() - x.mean()

    pooled = np.concatenate([x, y])
    n_total = pooled.size

    perm_stats = np.empty(n_permutations, dtype=np.float64)
    for i in range(n_permutations):
        rng.shuffle(pooled)
        x_perm = pooled[:n_x]
        y_perm = pooled[n_x:]
        perm_stats[i] = y_perm.mean() - x_perm.mean()

    p_val = np.mean(np.abs(perm_stats) >= np.abs(obs_stat))
    return float(obs_stat), float(p_val)

results = []

for layer_name in layer_keys:
    x = np.array(cos_pro[layer_name], dtype=np.float32)   
    y = np.array(cos_anti[layer_name], dtype=np.float32)
    assert x.shape == y.shape

    delta  = y - x
    delta2 = (y - x) * 2 / (y + x)

    n = int(delta.size)

    mean_delta_obs,  p_perm_delta  = permutation_pvalue_xy(x, y, n_permutations=10000)

    x2 = -delta2 / 2.0
    y2 = +delta2 / 2.0
    mean_delta2_obs, p_perm_delta2 = permutation_pvalue_xy(x2, y2, n_permutations=10000)

    results.append(
        {
            "layer": layer_name,
            "mean_delta":   float(delta.mean()),
            "p_perm_delta": float(p_perm_delta),
            "mean_delta2":  float(delta2.mean()),
            "p_perm_delta2": float(p_perm_delta2),
            "n_pairs": n,
        }
    )

res_df = pd.DataFrame(results)
res_df.to_csv("winobias_layerwise_permutation_tests_xy.csv", index=False)



In [ ]:
#explains the crazy bias in delta2
layer = "L21_norm"  # the one with -6.73

x = np.array(cos_pro[ "blocks.21.hook_normalized_resid_post"])
y = np.array(cos_anti["blocks.21.hook_normalized_resid_post"])

delta2 = (y - x) * 2 /(np.abs(y + x))
delta= y-x

print(delta2.min(), delta2.max())
print(np.quantile(delta2[~np.isnan(delta2)], [0.0, 0.01, 0.5, 0.99, 1.0]))
print(delta.min(), delta.max())

# expt for studying gender bias in hidden states in bert large using logits/ probability

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from scipy.stats import ttest_1samp, wilcoxon

from transformer_lens import HookedEncoder

In [ ]:
he_id  = model.tokenizer.convert_tokens_to_ids("he")
she_id = model.tokenizer.convert_tokens_to_ids("she")
mask_id = model.tokenizer.mask_token_id

layer_keys = (
    ["embed.hook_embed"]
    + [f"blocks.{l}.hook_resid_post" for l in range(model.cfg.n_layers)]
    + [f"blocks.{l}.hook_normalized_resid_post" for l in range(model.cfg.n_layers)]
    + ["mlm_head.ln.hook_normalized"]
)

print(f"he_id={he_id}, she_id={she_id}, mask_id={mask_id}")
print(f"Layer keys: {len(layer_keys)}")

In [ ]:
IGNORE_TOKENS = {"the", "a", "an"}
PRONOUNS = {"he", "she", "him", "her", "his", "hers"}

def parse_coref_indices(ex):
    """Parse Winobias coreference_clusters string list to int indices."""
    raw = ex["coreference_clusters"]
    return sorted({int(i) for i in raw})

In [ ]:
def preprocess_winobias(dataset, split_name):
    """
    Returns DataFrame with:
    - sentence_original
    - sentence_masked (with pronoun replaced by [MASK])
    - pron_pos (original token position of pronoun)
    - pron_word (original pronoun)
    - occ_word (profession word)
    - split
    """
    rows = []
    n_total, n_bad_coref, n_no_pronoun, n_bad_bert_tok = 0, 0, 0, 0

    for ex in dataset:
        n_total += 1
        words = ex["tokens"]
        words_l = [w.lower() for w in words]

        try:
            idxs = parse_coref_indices(ex)
        except Exception:
            n_bad_coref += 1
            continue

        # Find pronoun and non-determiner entity in cluster
        pron_pos, entity_pos = None, None
        for idx in idxs:
            if idx < 0 or idx >= len(words_l):
                continue
            w = words_l[idx]
            if w in PRONOUNS:
                if pron_pos is None:  # Take first pronoun
                    pron_pos = idx
            elif w not in IGNORE_TOKENS:
                if entity_pos is None:  # Take first non-determiner entity
                    entity_pos = idx

        if pron_pos is None or entity_pos is None:
            n_no_pronoun += 1
            continue

        pron_word = words[pron_pos]
        occ_word = words[entity_pos]

        # BERT tokenization: occupation must be single token
        occ_pieces = model.tokenizer.tokenize(occ_word)
        if len(occ_pieces) != 1:
            n_bad_bert_tok += 1
            continue

        # Create masked sentence: replace pronoun with [MASK]
        masked_words = words.copy()
        masked_words[pron_pos] = model.tokenizer.mask_token
        sentence_masked = " ".join(masked_words)
        sentence_orig = " ".join(words)

        rows.append({
            "split": split_name,
            "sentence_orig": sentence_orig,
            "sentence_masked": sentence_masked,
            "pron_pos": pron_pos,
            "pron_word": pron_word,
            "occ_word": occ_word,
            "occ_pos": entity_pos,
        })

    print(f"[{split_name}] total={n_total}, bad_coref={n_bad_coref}, "
          f"no_pronoun={n_no_pronoun}, bad_tok={n_bad_bert_tok}, kept={len(rows)}")
    return pd.DataFrame(rows)

In [ ]:
# Load and process all Winobias splits
print("Loading Winobias...")
type1_pro  = load_dataset("uclanlp/wino_bias", "type1_pro",  split="test")
type2_pro  = load_dataset("uclanlp/wino_bias", "type2_pro",  split="test")
df_type1_pro  = preprocess_winobias(type1_pro,  "type1_pro")
df_type2_pro  = preprocess_winobias(type2_pro,  "type2_pro")

winobias_df = pd.concat([
    df_type1_pro, 
    df_type2_pro
], ignore_index=True)

winobias_df.to_csv("winobias_masked_pronoun.csv", index=False)
print(f"Saved {len(winobias_df)} masked sentences")

In [ ]:
# Define gender pairs for Winobias pronouns
MALE_PRONOUNS   = {"he", "him", "his"}
FEMALE_PRONOUNS = {"she", "her", "hers"}


In [ ]:
import torch.nn.functional as F

In [ ]:
def get_pronoun_pair_ids(tokenizer, original_pron):
    """
    Returns (male_id, female_id) corresponding to the grammatical role
    of the original pronoun.
    """
    p = original_pron.lower()
    
    if p in ["he", "she"]:
        m, f = "he", "she"
    elif p in ["him", "her"]:

        m, f = "him", "her" 

    elif p in ["his", "hers"]:
        m, f = "his", "her" 
    else:
        # Fallback
        m, f = "he", "she"
    if p == "his":
        m, f = "his", "her"
    elif p == "hers":
        m, f = "his", "hers"

    m_id = tokenizer.convert_tokens_to_ids(m)
    f_id = tokenizer.convert_tokens_to_ids(f)
    return m_id, f_id

def batched_logit_analysis(winobias_df, batch_size=32):
    """
    For each sentence:
    - Identify correct male/female pronoun pair ids based on 'pron_word'
    - Compute logit(male) - logit(female) at mask position
    """
    N = len(winobias_df)
    results = []

    for start in range(0, N, batch_size):
        end = min(start + batch_size, N)
        batch = winobias_df.iloc[start:end]
        
        sentences = batch["sentence_masked"].tolist()
        pron_words = batch["pron_word"].tolist()
        occ_words  = batch["occ_word"].tolist()
        
        # Tokenize batch
        tokens, ttypes, amask = model.to_tokens(sentences)
        tokens = tokens.to(device)
        if ttypes is not None: ttypes = ttypes.to(device)
        if amask is not None:  amask = amask.to(device)

        # Forward pass
        logits, cache = model.run_with_cache(
            tokens,
            token_type_ids=ttypes,
            one_zero_attention_mask=amask,
            return_type="logits",
        )

        B = tokens.shape[0]
        
        # Prepare per-sample indices
        mask_positions = []
        male_ids = []
        female_ids = []
        
        for b in range(B):
            # Mask pos
            m_pos = (tokens[b] == mask_id).nonzero(as_tuple=False).item()
            mask_positions.append(m_pos)
            
            # Pronoun pair ids
            mid, fid = get_pronoun_pair_ids(model.tokenizer, pron_words[b])
            male_ids.append(mid)
            female_ids.append(fid)

        # Convert to tensors for gathering
        batch_idx = torch.arange(B, device=device)
        mask_pos_t = torch.tensor(mask_positions, device=device)
        male_ids_t = torch.tensor(male_ids, device=device)
        female_ids_t = torch.tensor(female_ids, device=device)

        # 1. True final logits
        # logits: [B, seq, vocab]
        final_logits_at_mask = logits[batch_idx, mask_pos_t, :] # [B, vocab]
        
        true_male_logits   = final_logits_at_mask.gather(1, male_ids_t.unsqueeze(1)).squeeze(1)
        true_female_logits = final_logits_at_mask.gather(1, female_ids_t.unsqueeze(1)).squeeze(1)
        true_diff = (true_male_logits - true_female_logits).detach().cpu().numpy()

        true_probs = F.softmax(final_logits_at_mask, dim=-1)
        true_male_probs   = true_probs.gather(1, male_ids_t.unsqueeze(1)).squeeze(1)
        true_female_probs = true_probs.gather(1, female_ids_t.unsqueeze(1)).squeeze(1)
        true_prob_diff = (true_male_probs - true_female_probs).detach().cpu().numpy()

        # 2. Logit lens per layer
        for k in layer_keys:
            resid = cache[k] # [B, seq, d_model]
            resid_at_mask = resid[batch_idx, mask_pos_t, :] # [B, d_model]
            
            layer_logits_all = resid_at_mask @ model.W_U + model.b_U # [B, vocab]
            
            l_male = layer_logits_all.gather(1, male_ids_t.unsqueeze(1)).squeeze(1)
            l_female = layer_logits_all.gather(1, female_ids_t.unsqueeze(1)).squeeze(1)
            layer_diff = (l_male - l_female).detach().cpu().numpy()

            layer_probs = F.softmax(layer_logits_all, dim=-1)
            p_male = layer_probs.gather(1, male_ids_t.unsqueeze(1)).squeeze(1)
            p_female = layer_probs.gather(1, female_ids_t.unsqueeze(1)).squeeze(1)
            layer_prob_diff = (p_male - p_female).detach().cpu().numpy()

            for i in range(B):
                results.append({
                    "split": batch.iloc[i]["split"],
                    "occ_word": occ_words[i],
                    "pron_word": pron_words[i],
                    "layer": k,
                    "true_logit_diff": true_diff[i],
                    "layer_logit_diff": layer_diff[i],
                    "logit_male": l_male[i].item(),
                    "logit_female": l_female[i].item(),

                    "true_prob_diff": true_prob_diff[i],
                    "layer_prob_diff": layer_prob_diff[i],
                    "prob_male": p_male[i].item(),
                    "prob_female": p_female[i].item(),
                })

        print(f"Processed {end}/{N} sentences")

    return pd.DataFrame(results)

# Run analysis
print("Running batched logit lens analysis...")
analysis_df = batched_logit_analysis(winobias_df)
analysis_df.to_csv("winobias_pronoun_mask_logit_bias.csv", index=False)
print("Saved winobias_pronoun_mask_logit_bias.csv")


In [ ]:
from scipy.stats import ttest_rel

In [ ]:
#  Occupation-wise stats
def occupation_bias_stats(analysis_df, layers_of_interest=None):
    """
    Groups by occ_word + layer, computes:
    - mean(logit(male) - logit(female)) per occupation
    - t-test p-value per occupation
    paired ttest on male v/s female
    """
    if layers_of_interest is None:
        layers_of_interest = layer_keys  # Default: final layer
        
    results = []
    
    for layer in layers_of_interest:
        layer_df = analysis_df[analysis_df["layer"] == layer]
        for occ_word, group in layer_df.groupby("occ_word"):
            logit_diffs = group["layer_logit_diff"].values
            prob_diffs  = group["layer_prob_diff"].values
            male_logits = group["logit_male"].values
            female_logits = group["logit_female"].values
            male_probs = group["prob_male"].values
            female_probs = group["prob_female"].values
            
            # Remove NaNs
            # clean_diffs = diffs[~np.isnan(diffs)]
            # n_clean = len(clean_diffs)
            valid_mask = ~(np.isnan(logit_diffs) | np.isnan(male_logits) | np.isnan(female_logits))
            logit_clean = logit_diffs[valid_mask]
            male_logit_clean = male_logits[valid_mask]
            female_logit_clean = female_logits[valid_mask]
            
            prob_valid_mask = ~(np.isnan(prob_diffs) | np.isnan(male_probs) | np.isnan(female_probs))
            prob_clean = prob_diffs[prob_valid_mask]
            male_prob_clean = male_probs[prob_valid_mask]
            female_prob_clean = female_probs[prob_valid_mask]

            n_samples = len(logit_clean)
            if n_samples < 2:
                continue

            mean_logit_diff = np.mean(logit_clean)
            mean_prob_diff = np.mean(prob_clean)
            t_logit_diff, p_logit_diff = ttest_1samp(logit_clean, popmean=0)
            t_prob_diff, p_prob_diff = ttest_1samp(prob_clean, popmean=0)
            
            # NEW: Paired t-tests (male vs female directly)
            t_logit_paired, p_logit_paired = ttest_rel(male_logit_clean, female_logit_clean)
            t_prob_paired, p_prob_paired = ttest_rel(male_prob_clean, female_prob_clean)
            
            
            
            results.append({
                "layer": layer,
                "occ_word": occ_word,
                "n_sentences": n_samples,

                # one sample t-tests
                "mean_logit_diff": mean_logit_diff,
                "t_logit_diff": t_logit_diff,
                "p_logit_diff": p_logit_diff,  # 1samp on differences
                "mean_prob_diff": mean_prob_diff,
                "t_prob_diff": t_prob_diff,
                "p_prob_diff": p_prob_diff,

                # Paired t-tests (NEW)
                "mean_logit_male": np.mean(male_logit_clean),
                "mean_logit_female": np.mean(female_logit_clean),
                "t_logit_paired": t_logit_paired,
                "p_logit_paired": p_logit_paired,  # paired male vs female logits
                "mean_prob_male": np.mean(male_prob_clean),
                "mean_prob_female": np.mean(female_prob_clean),
                "t_prob_paired": t_prob_paired,
                "p_prob_paired": p_prob_paired,   # paired male vs female probs

                "std_logit_diff": np.std(logit_clean),
                "std_prob_diff": np.std(prob_clean),
                
            })
    return pd.DataFrame(results)

In [ ]:
print("occupation bias stats with paired t-tests...")
occ_bias_stats = occupation_bias_stats(analysis_df, layer_keys)
occ_bias_stats.to_csv("winobias_occupation_wise_bias_logit_prob_paired.csv", index=False)

In [ ]:


print("\n=== UPDATED OCCUPATION BIAS STATS (1-SAMPLE + PAIRED T-TESTS) ===")
print(occ_bias_stats.round(4))

# Enhanced printing with both test types
for layer in layer_keys:
    layer_data = occ_bias_stats[occ_bias_stats["layer"] == layer]
    
    print(f"\n===== {layer} =====")
    
    print("\nTop 10 Male-biased (LOGIT DIFF, 1-SAMPLE T-TEST):")
    print(layer_data.nlargest(10, "mean_logit_diff")[["occ_word", "mean_logit_diff", "p_logit_diff", "n_sentences"]])
    
    print("\nTop 10 Male-biased (LOGIT PAIRED T-TEST):")
    print(layer_data.nlargest(10, "mean_logit_male")[
        ["occ_word", "mean_logit_male", "mean_logit_female", "p_logit_paired", "n_sentences"]
    ])
    
    print("\nTop 10 Female-biased (LOGIT DIFF, 1-SAMPLE T-TEST):")
    print(layer_data.nsmallest(10, "mean_logit_diff")[["occ_word", "mean_logit_diff", "p_logit_diff", "n_sentences"]])
    
    print("\nTop 10 Female-biased (LOGIT PAIRED T-TEST):")
    print(layer_data.nlargest(10, "mean_logit_female")[
        ["occ_word", "mean_logit_male", "mean_logit_female", "p_logit_paired", "n_sentences"]
    ])
    
    print("\nTop 10 Male-biased (PROB DIFF, 1-SAMPLE T-TEST):")
    print(layer_data.nlargest(10, "mean_prob_diff")[["occ_word", "mean_prob_diff", "p_prob_diff", "n_sentences"]])
    
    print("\nTop 10 Male-biased (PROB PAIRED T-TEST):")
    print(layer_data.nlargest(10, "mean_prob_male")[
        ["occ_word", "mean_prob_male", "mean_prob_female", "p_prob_paired", "n_sentences"]
    ])

# Summary statistics comparing both tests
print("\n=== TEST COMPARISON SUMMARY ===")
summary = occ_bias_stats.groupby("layer").agg({
    "p_logit_diff": ["mean", "median"],
    "p_logit_paired": ["mean", "median"],
    "p_prob_diff": ["mean", "median"],
    "p_prob_paired": ["mean", "median"],
}).round(4)
print(summary)


In [ ]:
import re

def layer_sort_key(layer):
    if layer.startswith("embed"):
        return -1
    m = re.search(r'blocks\.(\d+)\.', layer)
    if m:
        return int(m.group(1))
    return 1e9 



occ_bias_stats_prob = occ_bias_stats[
    occ_bias_stats["layer"] != "mlm_head.ln.hook_normalized"
]

pivot_prob = occ_bias_stats_prob.pivot(
    index="occ_word",
    columns="layer",
    values="mean_prob_diff"
).fillna(0)

ordered_layers = sorted(pivot_prob.columns, key=layer_sort_key)
pivot_prob = pivot_prob[ordered_layers]

plt.figure(figsize=(14, 10))
sns.heatmap(pivot_prob, center=0, cmap="RdBu_r", annot=False)
plt.title("Occupation Probability Bias: P(male) − P(female)")
plt.ylabel("Occupation")
plt.xlabel("Layer")
plt.tight_layout()
plt.savefig("occupation_bias_heatmap_probs.png", dpi=200)
plt.show()

# Top occupations by absolute bias (final layer)
final_layer_bias = occ_bias_stats[occ_bias_stats["layer"] == "mlm_head.ln.hook_normalized"]
final_layer_bias["abs_prob"] = np.abs(final_layer_bias["mean_prob_diff"])

print("\n=== STRONGEST BIAS (prob)(final layer) ===")
strongest = final_layer_bias.nlargest(20, "abs_prob")[["occ_word", "mean_prob_diff", "p_value_prob", "n_sentences"]]
print(strongest.round(4))


In [ ]:
import re

def layer_sort_key(layer):
    if layer.startswith("embed"):
        return -1
    m = re.search(r'blocks\.(\d+)\.', layer)
    if m:
        return int(m.group(1))
    return 1e9 



occ_bias_stats_prob = occ_bias_stats[
    occ_bias_stats["layer"] != "mlm_head.ln.hook_normalized"
]

pivot_prob = occ_bias_stats_prob.pivot(
    index="occ_word",
    columns="layer",
    values="mean_logit_diff"
).fillna(0)

ordered_layers = sorted(pivot_prob.columns, key=layer_sort_key)
pivot_prob = pivot_prob[ordered_layers]

plt.figure(figsize=(14, 10))
sns.heatmap(pivot_prob, center=0, cmap="RdBu_r", annot=False)
plt.title("Occupation Logit Bias: P(male) − P(female)")
plt.ylabel("Occupation")
plt.xlabel("Layer")
plt.tight_layout()
plt.savefig("occupation_bias_heatmap_logits_w_o_mlm.png", dpi=200)
plt.show()

In [ ]:
import re

def layer_sort_key(layer):
    if layer.startswith("embed"):
        return -1
    m = re.search(r'blocks\.(\d+)\.', layer)
    if m:
        return int(m.group(1))
    return 1e9 


pivot_prob = occ_bias_stats.pivot(
    index="occ_word",
    columns="layer",
    values="mean_logit_diff"
).fillna(0)

ordered_layers = sorted(pivot_prob.columns, key=layer_sort_key)
pivot_prob = pivot_prob[ordered_layers]

plt.figure(figsize=(14, 10))
sns.heatmap(pivot_prob, center=0, cmap="RdBu_r", annot=False)
plt.title("Occupation Logit Bias: P(male) − P(female)")
plt.ylabel("Occupation")
plt.xlabel("Layer")
plt.tight_layout()
plt.savefig("occupation_bias_heatmap_probs.png", dpi=200)
plt.show()

# Top occupations by absolute bias (final layer)
final_layer_bias = occ_bias_stats[occ_bias_stats["layer"] == "mlm_head.ln.hook_normalized"]
final_layer_bias["abs_bias"] = np.abs(final_layer_bias["mean_logit_diff"])

print("\n=== STRONGEST BIAS (final layer) ===")
strongest = final_layer_bias.nlargest(20, "abs_bias")[["occ_word", "mean_logit_diff", "p_value_logit", "n_sentences"]]
print(strongest.round(4))


# occupation wise plots

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import regex as re
import os

os.makedirs("bias_plots", exist_ok=True)

occ_bias_stats = pd.read_csv(
    "winobias_occupation_wise_bias_logit_prob.csv"
)

final_layer = "mlm_head.ln.hook_normalized"
final_occ_bias = occ_bias_stats[
    occ_bias_stats["layer"] == final_layer
].copy()

final_occ_bias["abs_bias"] = np.abs(final_occ_bias["mean_logit_diff"])

male_top5 = (
    final_occ_bias[final_occ_bias["mean_logit_diff"] > 0]
    .sort_values("abs_bias", ascending=False)
    .head(5)["occ_word"]
    .tolist()
)

female_top5 = (
    final_occ_bias[final_occ_bias["mean_logit_diff"] < 0]
    .sort_values("abs_bias", ascending=False)
    .head(5)["occ_word"]
    .tolist()
)

print("Top 5 Male:", male_top5)
print("Top 5 Female:", female_top5)



In [ ]:
def pval_column(value_col):
    return "p_value_logit" if value_col == "mean_logit_diff" else "p_value_prob"



In [ ]:
def annotate_pvals(bars, pvals, y_scale):
    for bar, p in zip(bars, pvals):
        if np.isnan(p):
            continue

        h = bar.get_height()
        offset = 0.03 * y_scale

        plt.text(
            bar.get_x() + bar.get_width() / 2,
            h + np.sign(h) * offset,
            f"p={p:.3f}",
            ha="center",
            va="bottom" if h >= 0 else "top",
            fontsize=9,
            rotation=90
        )


In [ ]:
def layer_index(layer):
    if "embed" in layer:
        return -1
    if "mlm_head" in layer:
        return 10_000
    m = re.search(r"blocks\.(\d+)\.", layer)
    return int(m.group(1)) if m else 9_999


def layer_short(layer):
    if "embed" in layer:
        return "Emb"
    if "mlm_head" in layer:
        return "Head"
    m = re.search(r"blocks\.(\d+)\.", layer)
    return f"L{m.group(1)}" if m else layer


def hook_type(layer):
    if "normalized" in layer:
        return "norm"
    return "raw"


In [ ]:
def plot_single_occupation(
    occ,
    bias_type,
    value_col,
    pval_col,
    title_suffix,
    fname_suffix,
):
    occ_data = occ_bias_stats[occ_bias_stats["occ_word"] == occ].copy()
    if occ_data.empty:
        return

    # Remove MLM head for probability plots
    if value_col == "mean_prob_diff":
        occ_data = occ_data[~occ_data["layer"].str.contains("mlm_head")]

    occ_data["layer_idx"] = occ_data["layer"].apply(layer_index)
    occ_data["layer_short"] = occ_data["layer"].apply(layer_short)
    occ_data["hook"] = occ_data["layer"].apply(hook_type)

    occ_data = occ_data.sort_values("layer_idx")

    layers = occ_data["layer_short"].unique()
    x = np.arange(len(layers))
    width = 0.35

    raw_vals, norm_vals = [], []
    raw_pvals, norm_pvals = [], []

    for l in layers:
        raw = occ_data[
            (occ_data["layer_short"] == l) & (occ_data["hook"] == "raw")
        ]
        norm = occ_data[
            (occ_data["layer_short"] == l) & (occ_data["hook"] == "norm")
        ]

        raw_vals.append(raw[value_col].values[0] if len(raw) else 0.0)
        norm_vals.append(norm[value_col].values[0] if len(norm) else 0.0)

        raw_pvals.append(raw[pval_col].values[0] if len(raw) else np.nan)
        norm_pvals.append(norm[pval_col].values[0] if len(norm) else np.nan)

    raw_vals = np.array(raw_vals)
    norm_vals = np.array(norm_vals)

    y_scale = np.max(np.abs(np.concatenate([raw_vals, norm_vals]))) + 1e-6

    plt.figure(figsize=(11, 6))

    bars_raw = plt.bar(
        x - width / 2,
        raw_vals,
        width,
        label="resid_post",
        alpha=0.8,
        edgecolor="black",
    )

    bars_norm = plt.bar(
        x + width / 2,
        norm_vals,
        width,
        label="resid_post_norm",
        alpha=0.8,
        edgecolor="black",
    )

    plt.axhline(0, color="black", linewidth=1)
    plt.xticks(x, layers, rotation=45)
    plt.ylabel(value_col)
    plt.xlabel("Layer")

    plt.title(f"{occ} ({bias_type})\n{title_suffix}", fontsize=13)
    plt.legend()

    annotate_pvals(bars_raw, raw_pvals, y_scale)
    annotate_pvals(bars_norm, norm_pvals, y_scale)

    plt.tight_layout()
    fname = f"bias_plots/{bias_type}_{occ}_{fname_suffix}.png"
    plt.savefig(fname, dpi=150)
    plt.show()
    plt.close()

    print(f"Saved {fname}")


In [ ]:
for occ in male_top5:
    plot_single_occupation(
        occ=occ,
        bias_type="logit",
        value_col="mean_logit_diff",
        pval_col="p_value_logit",
        title_suffix="Logit Bias (male − female)",
        fname_suffix="logits"
    )
    plot_single_occupation(
        occ=occ,
        bias_type="prob",
        value_col="mean_prob_diff",
        pval_col="p_value_prob",
        title_suffix="Probability Bias P(male) − P(female)",
        fname_suffix="probs"
    )

In [ ]:
for occ in female_top5:
    plot_single_occupation(
        occ=occ,
        bias_type="logit",
        value_col="mean_logit_diff",
        pval_col="p_value_logit",
        title_suffix="Logit Bias (male − female)",
        fname_suffix="logits"
    )
    plot_single_occupation(
        occ=occ,
        bias_type="prob",
        value_col="mean_prob_diff",
        pval_col="p_value_prob",
        title_suffix="Probability Bias P(male) − P(female)",
        fname_suffix="probs"
    )

In [ ]:
similar but plots both ttest rel and 1sampttest 
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# import regex as re
# import os

# os.makedirs("bias_plots", exist_ok=True)

# # Load the updated CSV with paired t-tests
# occ_bias_stats = pd.read_csv(
#     "winobias_occupation_wise_bias_logit_prob_paired.csv"  # Updated filename
# )

# final_layer = "mlm_head.ln.hook_normalized"
# final_occ_bias = occ_bias_stats[
#     occ_bias_stats["layer"] == final_layer
# ].copy()

# # Try both diff and paired metrics for top occupations
# final_occ_bias["abs_bias_diff"] = np.abs(final_occ_bias["mean_logit_diff"])
# final_occ_bias["abs_bias_paired"] = np.abs(final_occ_bias["mean_logit_male"] - final_occ_bias["mean_logit_female"])

# male_top5_diff = (
#     final_occ_bias[final_occ_bias["mean_logit_diff"] > 0]
#     .sort_values("abs_bias_diff", ascending=False)
#     .head(5)["occ_word"].tolist()
# )

# female_top5_diff = (
#     final_occ_bias[final_occ_bias["mean_logit_diff"] < 0]
#     .sort_values("abs_bias_diff", ascending=False)
#     .head(5)["occ_word"].tolist()
# )

# male_top5_paired = (
#     final_occ_bias[final_occ_bias["mean_logit_male"] > final_occ_bias["mean_logit_female"]]
#     .sort_values("abs_bias_paired", ascending=False)
#     .head(5)["occ_word"].tolist()
# )

# female_top5_paired = (
#     final_occ_bias[final_occ_bias["mean_logit_male"] < final_occ_bias["mean_logit_female"]]
#     .sort_values("abs_bias_paired", ascending=False)
#     .head(5)["occ_word"].tolist()
# )

# print("Top 5 Male (diff):", male_top5_diff)
# print("Top 5 Female (diff):", female_top5_diff)
# print("Top 5 Male (paired):", male_top5_paired)
# print("Top 5 Female (paired):", female_top5_paired)

# def pval_column(value_col):
#     """Map value column to correct p-value column"""
#     if value_col == "mean_logit_diff":
#         return "p_logit_diff"
#     elif value_col == "mean_logit_male":
#         return "p_logit_paired"
#     elif value_col == "mean_prob_diff":
#         return "p_prob_diff"
#     elif value_col == "mean_prob_male":
#         return "p_prob_paired"
#     return None

# def annotate_pvals(bars, pvals, y_scale):
#     for bar, p in zip(bars, pvals):
#         if np.isnan(p) or p > 0.05:  # Only annotate significant p-values
#             continue

#         h = bar.get_height()
#         offset = 0.03 * y_scale

#         sig_stars = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
#         plt.text(
#             bar.get_x() + bar.get_width() / 2,
#             h + np.sign(h) * offset,
#             sig_stars,
#             ha="center",
#             va="bottom" if h >= 0 else "top",
#             fontsize=12,
#             fontweight="bold",
#             color="red"
#         )

# def layer_index(layer):
#     if "embed" in layer:
#         return -1
#     if "mlm_head" in layer:
#         return 10_000
#     m = re.search(r"blocks\.(\d+)\.", layer)
#     return int(m.group(1)) if m else 9_999

# def layer_short(layer):
#     if "embed" in layer:
#         return "Emb"
#     if "mlm_head" in layer:
#         return "Head"
#     m = re.search(r"blocks\.(\d+)\.", layer)
#     return f"L{m.group(1)}" if m else layer

# def hook_type(layer):
#     if "normalized" in layer:
#         return "norm"
#     return "raw"

# def plot_single_occupation(
#     occ,
#     bias_type,
#     value_col,
#     test_type,
#     title_suffix,
#     fname_suffix,
# ):
#     """
#     bias_type: 'logits' or 'probs'
#     test_type: 'diff_1samp' or 'paired'
#     """
#     occ_data = occ_bias_stats[occ_bias_stats["occ_word"] == occ].copy()
#     if occ_data.empty:
#         print(f"No data for occupation: {occ}")
#         return

#     # Remove MLM head for probability plots
#     if "prob" in value_col:
#         occ_data = occ_data[~occ_data["layer"].str.contains("mlm_head")]

#     occ_data["layer_idx"] = occ_data["layer"].apply(layer_index)
#     occ_data["layer_short"] = occ_data["layer"].apply(layer_short)
#     occ_data["hook"] = occ_data["layer"].apply(hook_type)

#     occ_data = occ_data.sort_values("layer_idx")

#     layers = occ_data["layer_short"].unique()
#     x = np.arange(len(layers))
#     width = 0.35

#     raw_vals, norm_vals = [], []
#     raw_pvals, norm_pvals = [], []

#     pval_col = pval_column(value_col)

#     for l in layers:
#         raw = occ_data[
#             (occ_data["layer_short"] == l) & (occ_data["hook"] == "raw")
#         ]
#         norm = occ_data[
#             (occ_data["layer_short"] == l) & (occ_data["hook"] == "norm")
#         ]

#         raw_vals.append(raw[value_col].values[0] if len(raw) else 0.0)
#         norm_vals.append(norm[value_col].values[0] if len(norm) else 0.0)

#         raw_pvals.append(raw[pval_col].values[0] if len(raw) else np.nan)
#         norm_pvals.append(norm[pval_col].values[0] if len(norm) else np.nan)

#     raw_vals = np.array(raw_vals)
#     norm_vals = np.array(norm_vals)

#     y_scale = np.max(np.abs(np.concatenate([raw_vals, norm_vals]))) + 1e-6

#     plt.figure(figsize=(12, 6))

#     bars_raw = plt.bar(
#         x - width / 2,
#         raw_vals,
#         width,
#         label="resid_post",
#         alpha=0.8,
#         edgecolor="black",
#         color="skyblue"
#     )

#     bars_norm = plt.bar(
#         x + width / 2,
#         norm_vals,
#         width,
#         label="resid_post_norm",
#         alpha=0.8,
#         edgecolor="black",
#         color="salmon"
#     )

#     plt.axhline(0, color="black", linewidth=1.5)
#     plt.xticks(x, layers, rotation=45)
    
#     ylabel = "Logit Diff" if "logit" in value_col else "Prob Diff"
#     plt.ylabel(ylabel)
#     plt.xlabel("Layer")

#     test_label = "1-sample t-test\ndiff≠0" if test_type == "diff_1samp" else "Paired t-test\nmale≠female"
#     plt.title(f"{occ} ({bias_type} | {test_label})\n{title_suffix}", fontsize=13)
#     plt.legend()

#     annotate_pvals(bars_raw, raw_pvals, y_scale)
#     annotate_pvals(bars_norm, norm_pvals, y_scale)

#     plt.tight_layout()
#     fname = f"bias_plots/{bias_type}_{test_type}_{occ}_{fname_suffix}.png"
#     plt.savefig(fname, dpi=150, bbox_inches='tight')
#     plt.show()
#     plt.close()

#     print(f"Saved {fname}")

# # Generate plots for both test types and top occupations
# top_occupations = list(set(male_top5_diff + female_top5_diff + male_top5_paired + female_top5_paired))[:10]

# for occ in top_occupations:
#     # Logit plots - both test types
#     # plot_single_occupation(occ, "logits", "mean_logit_diff", "diff_1samp", 
#     #                       "1-sample t-test on logit(male)-logit(female)", "logits_diff")
#     plot_single_occupation(occ, "logits", "mean_logit_male", "paired", 
#                           "Paired t-test logit(male) vs logit(female)", "logits_paired")
    
#     # Probability plots - both test types  
#     # plot_single_occupation(occ, "probs", "mean_prob_diff", "diff_1samp", 
#     #                       "1-sample t-test on prob(male)-prob(female)", "probs_diff")
#     plot_single_occupation(occ, "probs", "mean_prob_male", "paired", 
#                           "Paired t-test prob(male) vs prob(female)", "probs_paired")

# print("✅ All paired t-test plots generated!")
# print("Check bias_plots/ directory for results.")


In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# import regex as re
# import os

# os.makedirs("bias_plots", exist_ok=True)

# # Load the updated CSV with paired t-tests
# occ_bias_stats = pd.read_csv(
#     "winobias_occupation_wise_bias_logit_prob_paired.csv"
# )

# final_layer = "mlm_head.ln.hook_normalized"
# final_occ_bias = occ_bias_stats[
#     occ_bias_stats["layer"] == final_layer
# ].copy()

# final_occ_bias["abs_bias_paired"] = np.abs(final_occ_bias["mean_logit_male"] - final_occ_bias["mean_logit_female"])

# male_top5_paired = (
#     final_occ_bias[final_occ_bias["mean_logit_male"] > final_occ_bias["mean_logit_female"]]
#     .sort_values("abs_bias_paired", ascending=False)
#     .head(5)["occ_word"].tolist()
# )

# female_top5_paired = (
#     final_occ_bias[final_occ_bias["mean_logit_male"] < final_occ_bias["mean_logit_female"]]
#     .sort_values("abs_bias_paired", ascending=False)
#     .head(5)["occ_word"].tolist()
# )

# print("Top 5 Male (paired):", male_top5_paired)
# print("Top 5 Female (paired):", female_top5_paired)

# def layer_index(layer):
#     if "embed" in layer:
#         return -1
#     if "mlm_head" in layer:
#         return 10_000
#     m = re.search(r"blocks\.(\d+)\.", layer)
#     return int(m.group(1)) if m else 9_999

# def layer_short(layer):
#     if "embed" in layer:
#         return "Emb"
#     if "mlm_head" in layer:
#         return "Head"
#     m = re.search(r"blocks\.(\d+)\.", layer)
#     return f"L{m.group(1)}" if m else layer

# def hook_type(layer):
#     if "normalized" in layer:
#         return "norm"
#     return "raw"

# def plot_paired_occupation(
#     occ,
#     bias_type,
#     title_suffix,
#     fname_suffix,
# ):
#     """
#     Plot PAIRED t-test results: Male vs Female logits/probs side-by-side per layer
#     4 bars per layer + 2 p-values per layer (one per hook)
#     """
#     occ_data = occ_bias_stats[occ_bias_stats["occ_word"] == occ].copy()
#     if occ_data.empty:
#         print(f"No data for occupation: {occ}")
#         return

#     # Remove MLM head for probability plots
#     if bias_type == "probs":
#         occ_data = occ_data[~occ_data["layer"].str.contains("mlm_head")]

#     occ_data["layer_idx"] = occ_data["layer"].apply(layer_index)
#     occ_data["layer_short"] = occ_data["layer"].apply(layer_short)
#     occ_data["hook"] = occ_data["layer"].apply(hook_type)

#     occ_data = occ_data.sort_values("layer_idx")

#     layers = occ_data["layer_short"].unique()
#     x = np.arange(len(layers))
#     width = 0.18

#     # Value columns for paired test
#     if bias_type == "logits":
#         male_col, female_col = "mean_logit_male", "mean_logit_female"
#         pval_col = "p_logit_paired"
#         ylabel = "Logits"
#     else:  # probs
#         male_col, female_col = "mean_prob_male", "mean_prob_female"
#         pval_col = "p_prob_paired"
#         ylabel = "Probabilities"

#     raw_male_vals, raw_female_vals = [], []
#     norm_male_vals, norm_female_vals = [], []
#     raw_pvals, norm_pvals = [], []

#     for l in layers:
#         # Raw layer
#         raw = occ_data[(occ_data["layer_short"] == l) & (occ_data["hook"] == "raw")]
#         raw_male_vals.append(raw[male_col].values[0] if len(raw) else 0.0)
#         raw_female_vals.append(raw[female_col].values[0] if len(raw) else 0.0)
#         raw_pvals.append(raw[pval_col].values[0] if len(raw) else np.nan)
        
#         # Norm layer
#         norm = occ_data[(occ_data["layer_short"] == l) & (occ_data["hook"] == "norm")]
#         norm_male_vals.append(norm[male_col].values[0] if len(norm) else 0.0)
#         norm_female_vals.append(norm[female_col].values[0] if len(norm) else 0.0)
#         norm_pvals.append(norm[pval_col].values[0] if len(norm) else np.nan)

#     # Convert to arrays
#     raw_male_vals = np.array(raw_male_vals)
#     raw_female_vals = np.array(raw_female_vals)
#     norm_male_vals = np.array(norm_male_vals)
#     norm_female_vals = np.array(norm_female_vals)
#     raw_pvals = np.array(raw_pvals)
#     norm_pvals = np.array(norm_pvals)

#     y_scale = np.max(np.abs(np.concatenate([raw_male_vals, raw_female_vals, norm_male_vals, norm_female_vals]))) + 1e-6

#     plt.figure(figsize=(14, 7))

#     # 4 bars per layer
#     bars_raw_male = plt.bar(x - 1.5*width, raw_male_vals, width, 
#                            label="Male (resid_post)", alpha=0.8, 
#                            edgecolor="black", color="skyblue")
#     bars_raw_female = plt.bar(x - 0.5*width, raw_female_vals, width, 
#                              label="Female (resid_post)", alpha=0.8, 
#                              edgecolor="black", color="lightcoral")
#     bars_norm_male = plt.bar(x + 0.5*width, norm_male_vals, width, 
#                             label="Male (resid_post_norm)", alpha=0.8, 
#                             edgecolor="black", color="steelblue")
#     bars_norm_female = plt.bar(x + 1.5*width, norm_female_vals, width, 
#                               label="Female (resid_post_norm)", alpha=0.8, 
#                               edgecolor="black", color="salmon")

#     plt.axhline(0, color="black", linewidth=1.5)
#     plt.xticks(x, layers, rotation=45)
#     plt.ylabel(ylabel)
#     plt.xlabel("Layer")
#     plt.title(f"{occ} ({bias_type} | Paired t-test: Male vs Female)\n{title_suffix}", fontsize=14)
#     plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

#     # Annotate 2 p-values per layer: one for raw (above raw male), one for norm (above norm male)
#     for i in range(len(layers)):
#         # Raw p-value (above raw male bar)
#         if not np.isnan(raw_pvals[i]):
#             h_raw = np.max([raw_male_vals[i], raw_female_vals[i]])  # Above highest raw bar
#             plt.text(
#                 x[i] - width,  # Center of raw male bar
#                 h_raw + 0.015 * y_scale,
#                 f"p={raw_pvals[i]:.3f}",
#                 ha="center",
#                 va="bottom",
#                 fontsize=9,
#                 fontweight="bold",
#                 rotation=45  # Slanted text
#             )
        
#         # Norm p-value (above norm male bar)
#         if not np.isnan(norm_pvals[i]):
#             h_norm = np.max([norm_male_vals[i], norm_female_vals[i]])  # Above highest norm bar
#             plt.text(
#                 x[i] + width,  # Center of norm male bar
#                 h_norm + 0.015 * y_scale,
#                 f"p={norm_pvals[i]:.3f}",
#                 ha="center",
#                 va="bottom",
#                 fontsize=9,
#                 fontweight="bold",
#                 rotation=45  # Slanted text
#             )

#     plt.tight_layout()
#     fname = f"bias_plots/{bias_type}_paired_4bar_{occ}_{fname_suffix}.png"
#     plt.savefig(fname, dpi=150, bbox_inches='tight')
#     plt.show()
#     plt.close()

#     print(f"Saved {fname}")

# # Generate plots for top occupations
# top_occupations = list(set(male_top5_paired + female_top5_paired))[:10]

# for occ in top_occupations:
#     plot_paired_occupation(occ, "logits", 
#                           "Paired t-test per hook: Male vs Female logits", "logits")
#     plot_paired_occupation(occ, "probs", 
#                           "Paired t-test per hook: Male vs Female probs", "probs")


# print("Check bias_plots/ directory for results.")


## permutation test for logit based occupaton wise gender bias



In [ ]:
import numpy as np
from scipy.stats import permutation_test
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def permutation_test_bias(analysis_df, occ_word, layer, bias_type='logits', n_permutations=5000, seed=42):
    """
    WEAT-style permutation test for occupation bias.
    """
    np.random.seed(seed)
    
    layer_df = analysis_df[(analysis_df["occ_word"] == occ_word) & (analysis_df["layer"] == layer)]
    
    if len(layer_df) < 4:
        return {'p_perm': np.nan, 'test_stat': np.nan, 'n_samples': 0,
                'mean_male': np.nan, 'mean_female': np.nan}
    
    if bias_type == 'logits':
        male_vals = layer_df["logit_male"].dropna().values
        female_vals = layer_df["logit_female"].dropna().values
    else:
        male_vals = layer_df["prob_male"].dropna().values
        female_vals = layer_df["prob_female"].dropna().values
    
    n_male, n_female = len(male_vals), len(female_vals)
    n_samples = n_male + n_female
    
    if n_male < 2 or n_female < 2:
        return {'p_perm': np.nan, 'test_stat': np.nan, 'n_samples': n_samples,
                'mean_male': np.mean(male_vals) if n_male else np.nan,
                'mean_female': np.mean(female_vals) if n_female else np.nan}
    
    observed_stat = np.mean(male_vals) - np.mean(female_vals)
    
    # Efficient vectorized permutations
    permuted_stats = np.zeros(n_permutations)
    all_data = np.concatenate([male_vals, female_vals])
    
    for i in range(n_permutations):
        np.random.shuffle(all_data)
        perm_male = all_data[:n_male]
        perm_female = all_data[n_male:]
        permuted_stats[i] = np.mean(perm_male) - np.mean(perm_female)
    
    # Two-tailed p-value
    p_value = np.mean(np.abs(permuted_stats) >= np.abs(observed_stat))
    
    return {
        'p_perm': p_value,
        'test_stat': observed_stat,
        'n_samples': n_samples,
        'n_male': n_male,
        'n_female': n_female,
        'mean_male': np.mean(male_vals),
        'mean_female': np.mean(female_vals),
        'perm_std': np.std(permuted_stats)
    }

def occupation_permutation_stats(analysis_df, layers_of_interest=None, n_permutations=5000):
    """
    Sequential version - no multiprocessing issues
    """
    if layers_of_interest is None:
        layers_of_interest = analysis_df["layer"].unique()
    
    results = []
    unique_combos = analysis_df[["occ_word", "layer"]].drop_duplicates()
    
    print(f"Processing {len(unique_combos)} occupation-layer combos...")
    
    for idx, (occ_word, layer) in enumerate(unique_combos.itertuples(index=False)):
        if idx % 50 == 0:
            print(f"Progress: {idx}/{len(unique_combos)} ({idx/len(unique_combos)*100:.1f}%)")
        
        # Logits
        logit_results = permutation_test_bias(analysis_df, occ_word, layer, 'logits', n_permutations)
        logit_results.update({'layer': layer, 'occ_word': occ_word, 'bias_type': 'logits'})
        results.append(logit_results)
        
        # Probs
        prob_results = permutation_test_bias(analysis_df, occ_word, layer, 'probs', n_permutations)
        prob_results.update({'layer': layer, 'occ_word': occ_word, 'bias_type': 'probs'})
        results.append(prob_results)
    
    results_df = pd.DataFrame(results)
    print("✅ Permutation tests complete!")
    return results_df




In [ ]:
print("Running permutation tests...")
perm_stats = occupation_permutation_stats(analysis_df, n_permutations=2000)  
perm_stats.to_csv("winobias_permutation_test_results.csv", index=False)

print("\n=== TOP 10 MALE-BIASED LOGITS (PERMUTATION TEST) ===")
logit_perm = perm_stats[(perm_stats['bias_type']=='logits')]
print(logit_perm.nlargest(10, 'test_stat')[
    ['occ_word', 'layer', 'test_stat', 'p_perm', 'n_samples', 'mean_male', 'mean_female']
].round(4))

print("\n=== TOP 10 FEMALE-BIASED LOGITS ===")
female_bias = logit_perm.nsmallest(10, 'test_stat')
print(logit_perm.nsmallest(10, 'test_stat')[
    ['occ_word', 'layer', 'test_stat', 'p_perm', 'n_samples', 'mean_male', 'mean_female']
].round(4))

# Compare with t-test p-values 
if 'occ_bias_stats' in globals():
    final_layer = "mlm_head.ln.hook_normalized"
    comparison = occ_bias_stats[occ_bias_stats["layer"] == final_layer].merge(
        perm_stats[(perm_stats['layer']==final_layer) & (perm_stats['bias_type']=='logits')],
        on=['layer', 'occ_word']
    )
    print("\n=== T-TEST vs PERMUTATION (Final Layer) ===")
    print(comparison.nlargest(10, 'test_stat')[
        ['occ_word', 'mean_logit_male', 'p_logit_paired', 'p_perm']
    ].round(4))
